In [1]:
import pandas as pd

In [2]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
import seaborn as sns

In [4]:
import sys
sys.path.append('/home/anastasia/anaconda3/envs/just3/lib/python3.7/site-packages')
sys.path.append('/home/anastasia/anaconda3/envs/just3/lib/python3.7')

In [5]:
import lightgbm  as lgb

In [6]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit , KFold

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score

In [9]:
from sklearn import metrics

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv('_usa_.csv')

Разделение на train/test

1) 80/20

In [47]:
train = df.head(int(0.8 * len(df)))
test = df.tail(int(0.2 * len(df))+1)

y_test = test['IsFraud']

test.drop('IsFraud', axis='columns', inplace=True)

X_train = train.drop('IsFraud', axis='columns').copy()
X_test = test.copy()
y_train = train['IsFraud'].copy()

2) split

In [14]:
X = df.drop('IsFraud', axis='columns').copy()
y = df['IsFraud'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, stratify = y, random_state = 0)

3) 80/20 по клиенту

In [26]:
test = pd.DataFrame()
train = pd.DataFrame()

for name in df.columns:
    test[name] = []
    train[name] = []

for acc in df['Card Number'].unique():
    df_tmp = df.loc[df['Card Number'] == acc]
    li = df_tmp['TransactionId'].to_list()
    a = int(len(li) // 8 + 1)
    test = test.append(df_tmp.tail(a), ignore_index = True)
    train = train.append(df_tmp.head(len(li) - a), ignore_index = True)

y_test = test['IsFraud']

test.drop('IsFraud', axis='columns', inplace=True)

X_train = train.drop('IsFraud', axis='columns').copy()
X_test = test.copy()
y_train = train['IsFraud'].copy()

Валидация

In [16]:
#x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size = 0.15, stratify = y_train, random_state = 0)

In [18]:
n_folds = 5
folds = TimeSeriesSplit(n_splits=n_folds) 
folds = KFold(n_splits = 5)

In [20]:
columns = X_train.columns
splits = folds.split(X_train, y_train)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])

In [21]:
score_auc = 0 

In [22]:
us = {}
us['objective'] = 'binary'
us['boosting_type'] = 'gbdt'
us['metric'] = 'auc'
us['learning_rate'] = 0.006
us['num_leaves'] = 60
us['max_depth'] = 5
us['feature_fraction'] = 0.04

us['is_unbalance'] = True
us['bagging_fraction'] = 0.33
us['min_data_in_leaf'] = 80
us['min_child_weight'] = 0.03
us['bagging_seed'] = 7
us['verbosity'] = -1
us['reg_alpha'] = 0.3
us['reg_lambda'] = 0.6
us['random_state'] = 0

In [23]:
%%time

for fold_n, (train_index, valid_index) in enumerate(splits):
    X_tr, X_val = X_train[columns].iloc[train_index], X_train[columns].iloc[valid_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[valid_index]

    dtrain = lgb.Dataset(X_tr, label = y_tr)
    dvalid = lgb.Dataset(X_val, label = y_val)
    
    clf = lgb.train(us, dtrain, 1000000, valid_sets = [dtrain, dvalid], verbose_eval = 400, early_stopping_rounds = 4000)
    
    y_pred_val = clf.predict(X_val)
    y_oof[valid_index] = y_pred_val
    print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_val, y_pred_val)}")
    
    score_auc += roc_auc_score(y_val, y_pred_val) / n_folds
    
    y_preds += clf.predict(X_test) / n_folds
    
    del X_tr, X_val, y_tr, y_val
 

Training until validation scores don't improve for 4000 rounds
[400]	training's auc: 0.999822	valid_1's auc: 0.99914
[800]	training's auc: 0.999862	valid_1's auc: 0.999277
[1200]	training's auc: 0.999902	valid_1's auc: 0.999354
[1600]	training's auc: 0.999932	valid_1's auc: 0.999456
[2000]	training's auc: 0.999949	valid_1's auc: 0.999523
[2400]	training's auc: 0.999963	valid_1's auc: 0.999607
[2800]	training's auc: 0.999975	valid_1's auc: 0.999657
[3200]	training's auc: 0.999986	valid_1's auc: 0.999682
[3600]	training's auc: 0.999995	valid_1's auc: 0.999707
[4000]	training's auc: 1	valid_1's auc: 0.999732
[4400]	training's auc: 1	valid_1's auc: 0.999744
[4800]	training's auc: 1	valid_1's auc: 0.999752
[5200]	training's auc: 1	valid_1's auc: 0.999764
[5600]	training's auc: 1	valid_1's auc: 0.999759
[6000]	training's auc: 1	valid_1's auc: 0.999749
[6400]	training's auc: 1	valid_1's auc: 0.999761
[6800]	training's auc: 1	valid_1's auc: 0.999754
[7200]	training's auc: 1	valid_1's auc: 0.99

[3200]	training's auc: 0.999982	valid_1's auc: 0.999744
[3600]	training's auc: 0.999987	valid_1's auc: 0.999753
[4000]	training's auc: 0.999991	valid_1's auc: 0.99977
[4400]	training's auc: 0.999993	valid_1's auc: 0.999778
[4800]	training's auc: 0.999994	valid_1's auc: 0.999785
[5200]	training's auc: 0.999996	valid_1's auc: 0.999797
[5600]	training's auc: 0.999997	valid_1's auc: 0.999802
[6000]	training's auc: 0.999999	valid_1's auc: 0.999795
[6400]	training's auc: 1	valid_1's auc: 0.999797
[6800]	training's auc: 1	valid_1's auc: 0.999804
[7200]	training's auc: 1	valid_1's auc: 0.999804
[7600]	training's auc: 1	valid_1's auc: 0.999804
[8000]	training's auc: 1	valid_1's auc: 0.999806
[8400]	training's auc: 1	valid_1's auc: 0.999806
[8800]	training's auc: 1	valid_1's auc: 0.999812
[9200]	training's auc: 1	valid_1's auc: 0.99981
[9600]	training's auc: 1	valid_1's auc: 0.999812
[10000]	training's auc: 1	valid_1's auc: 0.99981
[10400]	training's auc: 1	valid_1's auc: 0.999815
[10800]	traini

In [24]:
print(f"\nMean AUC = {score_auc}")
print(f"\nOut of folds AUC= {roc_auc_score(y_train, y_oof)}")
print(f"\nTest AUC = {roc_auc_score(y_test, y_preds)}")



Mean AUC = 0.9953567316862428

Out of folds AUC= 0.9979993142508988

Test AUC = 0.9986724982852015


In [30]:
y_pred = y_preds


for i in range(len(y_preds)): 
    if (y_preds[i] >= .5):
        y_pred[i] = 1
    else:
        y_pred[i] = 0

In [31]:
print(f"\nOut of folds confusion matrix \n {metrics.confusion_matrix(y_train, y_oof)}\n")
print(f"\nTest confusion matrix \n {metrics.confusion_matrix(y_preds, y_test)}")


Out of folds confusion matrix 
 [[80354   112]
 [    9   147]]


Test confusion matrix 
 [[14194     5]
 [    7    22]]
